In [80]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

OverflowError: Python int too large to convert to C long

In [1]:
import pandas as pd
from pathlib import Path

def ler_txt_escriturador(path_txt: str | Path, encoding: str = "latin1") -> pd.DataFrame:
    path_txt = Path(path_txt)

    # 1) Lê todas as linhas
    with path_txt.open("r", encoding=encoding, errors="replace") as f:
        linhas = [ln.rstrip("\n") for ln in f]

    # 2) Acha a linha de cabeçalho "real"
    idx_header = next(
        (i for i, ln in enumerate(linhas) if ln.startswith("Data do Movimento;")),
        None
    )
    if idx_header is None:
        raise ValueError("Não encontrei a linha de cabeçalho que começa com 'Data do Movimento;'")

    header_line = linhas[idx_header]

    # 3) Seleciona apenas as linhas de dados (as que começam com data dd/mm/aaaa)
    #    (ignora abertura e footer técnico)
    data_lines = [
        ln for ln in linhas[idx_header + 1:]
        if len(ln) >= 10 and ln[2] == "/" and ln[5] == "/"  # padrão simples dd/mm/yyyy
    ]

    # 4) Monta um CSV em memória (cabeçalho + dados) e lê com pandas
    from io import StringIO
    csv_text = header_line + "\n" + "\n".join(data_lines)

    df = pd.read_csv(
        StringIO(csv_text),
        sep=";",
        dtype=str,          # lê tudo como texto primeiro (evita bagunça com CPF, CEP etc.)
        engine="python"
    )

    # 5) Remove colunas vazias criadas por ';' no final das linhas
    df = df.dropna(axis=1, how="all")

    # 6) (Opcional) Converter colunas numéricas com vírgula decimal
    # Exemplo: "Quantidade de Ativos" vem como "5549390,000"
    if "Quantidade de Ativos" in df.columns:
        df["Quantidade de Ativos"] = (
            df["Quantidade de Ativos"]
            .str.replace(".", "", regex=False)     # se tiver separador de milhar
            .str.replace(",", ".", regex=False)    # vírgula -> ponto
        )
        df["Quantidade de Ativos"] = pd.to_numeric(df["Quantidade de Ativos"], errors="coerce")

    # 7) (Opcional) Converter datas principais
    if "Data do Movimento" in df.columns:
        df["Data do Movimento"] = pd.to_datetime(df["Data do Movimento"], dayfirst=True, errors="coerce")
    if "Data de Nascimento / Fundação" in df.columns:
        df["Data de Nascimento / Fundação"] = pd.to_datetime(df["Data de Nascimento / Fundação"], errors="coerce")

    return df


In [75]:
df = ler_txt_escriturador("teste11_parte_14.txt")
print(df.head())
print(df.shape)


ParserError: field larger than field limit (131072)

In [71]:
df.to_excel("saldo_teste11_parte13.xlsx", index=False)


In [29]:
def contar_linhas(path_txt, encoding="latin1"):
    with open(path_txt, "r", encoding=encoding, errors="ignore") as f:
        return sum(1 for _ in f)


In [30]:
def dividir_txt_em_duas_partes(
    path_txt,
    output_1,
    output_2,
    encoding="latin1"
):
    total_linhas = contar_linhas(path_txt, encoding)
    metade = total_linhas // 2

    with open(path_txt, "r", encoding=encoding, errors="ignore") as fin, \
         open(output_1, "w", encoding=encoding) as f1, \
         open(output_2, "w", encoding=encoding) as f2:

        for i, linha in enumerate(fin):
            if i < metade:
                f1.write(linha)
            else:
                f2.write(linha)


In [68]:
dividir_txt_em_duas_partes(
    "teste11_parte_12.txt",
    "teste11_parte_13.txt",
    "teste11_parte_14.txt"
)


In [2]:
import pandas as pd
import csv
from pathlib import Path

def ler_txt_escriturador_grande_sem_aspas(path_txt, encoding="latin1"):
    path_txt = Path(path_txt)

    # acha header
    with path_txt.open("r", encoding=encoding, errors="replace") as f:
        idx_header = None
        for i, ln in enumerate(f):
            if ln.startswith("Data do Movimento;"):
                idx_header = i
                break
    if idx_header is None:
        raise ValueError("Não encontrei a linha de cabeçalho 'Data do Movimento;'")

    df = pd.read_csv(
        path_txt,
        sep=";",
        skiprows=idx_header,
        header=0,
        dtype=str,
        encoding=encoding,
        engine="c",
        quoting=csv.QUOTE_NONE,   # <- IGNORA aspas
        on_bad_lines="skip"
    )
    df = df.dropna(axis=1, how="all")
    return df



In [3]:
df = ler_txt_escriturador_grande_sem_aspas("teste11_ler_txt.txt")
print(df.head())
print(df.shape)

  Data do Movimento Código do Escriturador CPF/CNPJ do Investidor  \
0        08/01/2026                    359            99716950900   
1        08/01/2026                    359            99719053615   
2        08/01/2026                    359            99724286134   
3        08/01/2026                    359            99728788720   
4        08/01/2026                    359            99730359504   

  Data de Nascimento / Fundação Código de Dependência  \
0                    1974-11-27                     1   
1                    1973-10-10                     1   
2                    1984-02-06                     1   
3                    1967-05-16                     1   
4                    1980-12-07                     1   

                                  Nome do Investidor Tipo de Pessoa  \
0  MARCELO RODRIGUES DE OLIVEIRA                 ...              F   
1  PAULO ROBERTO SILVA                           ...              F   
2  HELISANGELA NAVARRO DOS SA

In [4]:
df.to_excel("saldo_teste11_.xlsx", index=False)